# Visual Data Analysis of Fraudulent Transactions


In [12]:
# initial imports
!pip install sqlalchemy
!pip install plotly
!pip install pymysql
!pip install hvplot


import pandas as pd
import calendar
import plotly.express as px
import hvplot.pandas
from sqlalchemy import create_engine
import datetime


In [13]:
#create a connection to the database

username = 'root'
password = 'Ahasan321!'
hostname = 'localhost'
database = 'newschema'

# MySQL connection string
engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}/{database}")


In [14]:
# loading data from the database

def execute_query(query):

    transaction_df = pd.read_sql(sql=query, con=engine, index_col='DATE', parse_dates='DATE')

    return transaction_df

def fraud_transactions(df):
    start_time = datetime.time(7,0,0)
    end_time = datetime.time(9,0,0)
    return df.between_time(start_time, end_time).sort_values('AMOUNT', ascending=False)
    

In [15]:
# loading data for card holder 2 and 18 from the database
query = f"""
SELECT a.ID, a.Name, b.Card, c.DATE, c.AMOUNT, e.NAME as 'category'
FROM CardHolder a, CreditCard b, Transaction c, Merchant d, MerchantCategory e
WHERE a.ID = b.IDCardHolder AND b.Card = c.CARD AND c.IDMerchant = d.ID AND d.IDMerchantCategory = e.ID
"""
transaction_df = execute_query(query)

suspect_df = fraud_transactions(transaction_df).head(100)
suspect_df

,ID,Name,Card,AMOUNT,category
DATE,,,,,
2018-12-07 07:22:03,1,Robert Johnson,4761049645711555811,1894.00,bar
2018-03-05 08:26:08,16,Crystal Clark,5570600642865857,1617.00,bar
2018-03-06 07:18:09,25,Nancy Contreras,4319653513507,1334.00,bar
2018-01-22 08:07:03,16,Crystal Clark,5570600642865857,1131.00,restaurant
2018-09-26 08:48:40,1,Robert Johnson,4761049645711555811,1060.00,restaurant
...,...,...,...,...,...
2018-04-08 07:06:20,13,John Martin,3561954487988605,11.73,food truck
2018-09-15 08:33:49,16,Crystal Clark,5500708021555307,11.72,coffee shop
2018-12-18 07:45:28,21,Dana Washington,4279104135293225293,11.70,food truck


In [16]:
px.scatter(suspect_df, x=suspect_df.index, y='AMOUNT', color='category', size='AMOUNT', hover_data=['Name', 'category'], labels = {'x' : 'DATE', 'y' : 'AMOUNT'}, title='Early Hour Transactions')


Data Analysis Questions 1

Use hvPlot to create a line plot showing a time series from the transactions along all the year for card holders 2 and 18. In order to contrast the patterns of both card holders, create a line plot containing both lines. What difference do you observe between the consumption patterns? Does the difference could be a fraudulent transaction? Explain your rationale

In [17]:
# loading data for card holder 2 and 18 from the database
query = f"""
SELECT a.ID, a.Name, b.Card, c.DATE, c.AMOUNT, e.NAME as 'category'
FROM CardHolder a, CreditCard b, Transaction c, Merchant d, MerchantCategory e
WHERE a.ID = b.IDCardHolder AND b.Card = c.CARD AND c.IDMerchant = d.ID AND d.IDMerchantCategory = e.ID
"""
transaction_df = execute_query(query)
transaction_df.head()

data_by_id = transaction_df.groupby('ID')
data_by_id


In [18]:
# plot for cardholder 2

cardholder_2 = data_by_id.get_group(2)['AMOUNT'].hvplot(ylabel='AMOUNT', xlabel='DATE', title="Transactions by Id Holder 2")
cardholder_2

:Curve   [DATE]   (AMOUNT)

In [19]:
# plot for cardholder 18

cardholder_18 = data_by_id.get_group(18)['AMOUNT'].hvplot(ylabel='AMOUNT', xlabel='DATE', title="Transactions by Id Holder 18")
cardholder_18

:Curve   [DATE]   (AMOUNT)

Conclusions for Question 1
The consumption pattern for both the id holder is very different. Id Holder 2 makes too many small transactions. Id Holder 18 has transactions ranging till $1839. Id Holder 2 is more suspectable to fraudulent transactions



Data Analysis Question 2

Use Plotly Express to create a series of six box plots, one for each month, in order to identify how many outliers could be per month for card holder id 25. By observing the consumption patters, do you see any anomalies? Write your own conclusions about your insights.

In [22]:
# loading data of daily transactions from jan to jun 2018 for card holder 25
card_holder_id = 25
start_date = '2018-01-01' 
end_date = '2018-07-01' 
query = f"""
SELECT a.ID, a.Name, b.Card, c.DATE, c.AMOUNT, e.NAME as 'Category'
FROM CardHolder a, CreditCard b, Transaction c, Merchant d, MerchantCategory e
WHERE a.ID = b.IDCardHolder AND b.Card = c.CARD AND c.IDMerchant = d.ID AND d.IDMerchantCategory = e.ID
AND c.DATE BETWEEN '2018-01-01 00:00:00' AND '2018-07-01 00:00:00'
AND a.ID = 25
"""

transaction_df = execute_query(query)
transaction_df.head()


,ID,Name,Card,AMOUNT,Category
DATE,,,,,
2018-04-02 01:50:15,25,Nancy Contreras,372414832802279,7.08,food truck
2018-04-18 10:12:40,25,Nancy Contreras,372414832802279,7.39,bar
2018-01-21 23:04:02,25,Nancy Contreras,372414832802279,2.22,food truck
2018-06-30 03:05:55,25,Nancy Contreras,372414832802279,2.27,bar
2018-04-26 02:16:45,25,Nancy Contreras,372414832802279,2.79,bar


In [23]:
# change the numeric month to month names
transaction_df['Month'] = transaction_df.index.month_name()
transaction_df.head()


,ID,Name,Card,AMOUNT,Category,Month
DATE,,,,,,
2018-04-02 01:50:15,25,Nancy Contreras,372414832802279,7.08,food truck,April
2018-04-18 10:12:40,25,Nancy Contreras,372414832802279,7.39,bar,April
2018-01-21 23:04:02,25,Nancy Contreras,372414832802279,2.22,food truck,January
2018-06-30 03:05:55,25,Nancy Contreras,372414832802279,2.27,bar,June
2018-04-26 02:16:45,25,Nancy Contreras,372414832802279,2.79,bar,April


In [24]:
# creating the six box plots using plotly express
px.box(transaction_df, y='AMOUNT', hover_data=['Category','Card'],x='Month', color='Category', title='Transactions by Id Holder 25')


Conclusions for Question 2
There seems to be fraudulent transactions pertaining to Restaurant & Food Truck category where Food Truck is ranging from 
1046